клонируем репо SSD + ставим зависимости
bash
Копировать
Редактировать


In [1]:
# %%bash
!pip install -q tqdm pillow-simd pycocotools
# + PyTorch/torchvision уже стоят в Colab
!echo "✓ libs installed"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
✓ libs installed


скачиваем BCCD‑датасет

In [2]:
# %%bash
!git clone https://github.com/Shenggan/BCCD_Dataset.git bccd_raw
!echo "✓ dataset downloaded"


Cloning into 'bccd_raw'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 29.55 MiB/s, done.
Resolving deltas: 100% (378/378), done.
✓ dataset downloaded


label‑map и вспомогательные функции

In [3]:
%%writefile utils_bccd.py
label_map = {'background': 0,
             'RBC': 1,
             'WBC': 2,
             'Platelets': 3}
rev_label_map = {v: k for k, v in label_map.items()}

Writing utils_bccd.py


create_data_lists_bccd.py
(сплит 70 / 15 / 15   train/val/test)

In [8]:
%%writefile create_data_lists_bccd.py
"""
Генерирует три JSON‑файла:
  data_bccd/BCCD_train.json
  data_bccd/BCCD_val.json
  data_bccd/BCCD_test.json
в формате, который ждёт оригинальный train.py из репозитория SSD.
"""
import sys, pathlib
sys.path.append('/content/ssd')   # путь к корню проекта
import json, random, xml.etree.ElementTree as ET
from pathlib import Path
from utils_bccd import label_map

root = Path('bccd_raw/BCCD')
imgs   = sorted((root / 'JPEGImages').glob('*.[jJpP][pPnN][gG]'))
annots = sorted((root / 'Annotations').glob('*.xml'))
assert len(imgs) == len(annots), "число картинок и xml не совпадает"

idx = list(range(len(imgs)))
random.shuffle(idx)

n = len(idx)
n_train = int(0.7 * n)
n_val   = int(0.15 * n)

splits = {
    'train': idx[:n_train],
    'val'  : idx[n_train:n_train+n_val],
    'test' : idx[n_train+n_val:],
}

def export(name, indices):
    images, objects = [], []
    for i in indices:
        img_p = imgs[i]
        xml_p = annots[i]

        boxes, labels, diff = [], [], []
        tree = ET.parse(xml_p).getroot()
        for obj in tree.iter('object'):
            cls = obj.find('name').text
            if cls not in label_map:          # пропускаем лишние классы
                continue

            bnd = obj.find('bndbox')
            xmin = int(float(bnd.find('xmin').text))
            ymin = int(float(bnd.find('ymin').text))
            xmax = int(float(bnd.find('xmax').text))
            ymax = int(float(bnd.find('ymax').text))

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label_map[cls])
            diff.append(0)                    # BCCD — все объекты «легкие»

        images.append(str(img_p.resolve()))
        objects.append({'boxes': boxes,
                        'labels': labels,
                        'difficulties': diff})

    out = Path('ssd/data_bccd')
    out.mkdir(exist_ok=True)

    with open(out / f'{name.upper()}_images.json', 'w') as f:
      json.dump(images, f)

    with open(out / f'{name.upper()}_objects.json', 'w') as f:
      json.dump(objects, f)
    print(f'✓ {name}: {len(images)} imgs')

for split_name, split_idx in splits.items():
    export(split_name, split_idx)


Overwriting create_data_lists_bccd.py


In [9]:
!python create_data_lists_bccd.py

✓ train: 254 imgs
✓ val: 54 imgs
✓ test: 56 imgs


In [10]:
%%writefile train_bccd.py
import subprocess, sys, pathlib

root = pathlib.Path('.').resolve()   # текущая папка Colab
data_folder = root/'data_bccd'
ckpt_dir    = root/'checkpoints'
ckpt_dir.mkdir(exist_ok=True)

cmd = [
    sys.executable,
    '/content/ssd/train.py',              # абсолютный путь
    '--data_folder', '/content/ssd/data_bccd',
    '--label_map',   '/content/utils_bccd.py',
    '--checkpoint',  '/content/checkpoints/ssd300_bccd.pth',
    '--num_epochs',  '25',
    '--batch_size',  '8',
    '--validation',
]

print(' '.join(cmd))
subprocess.run(cmd, check=True)


Overwriting train_bccd.py


In [11]:
%%writefile detect_bccd.py
import argparse, pathlib, json, subprocess, sys

parser = argparse.ArgumentParser()
parser.add_argument('--data_folder', default='ssd/data_bccd', type=str)
parser.add_argument('--trained_model', required=True,        type=str)
parser.add_argument('--label_map',     default='utils_bccd.py')
parser.add_argument('--min_score',     default='0.2')
parser.add_argument('--max_overlap', default='0.35')
parser.add_argument('--top_k', default='35')
args = parser.parse_args()

# путь к TEST_images.json
images_path = pathlib.Path(args.data_folder) / 'TEST_images.json'
test_images = json.load(open(images_path))

if not test_images:
    raise ValueError("❌ TEST_images.json пустой или не найдено изображений.")

img_path = pathlib.Path(test_images[0])  # ← путь к первой картинке

# команда вызова detect.py
cmd = [
    sys.executable,
    '/content/ssd/detect.py',
    '--trained_model', args.trained_model,
    '--input',         str(img_path),
    '--label_map',     args.label_map,
    '--min_score',     args.min_score,
    '--max_overlap', args.max_overlap,
    '--top_k', args.top_k,
]

print('👉 Запускаем:\n', ' '.join(cmd))
subprocess.run(cmd, check=True)


Overwriting detect_bccd.py


In [29]:

!python train_bccd.py      # долгая ячейка, ~10‑15 мин на Colab GPU

# показывает картинку с найденными клетками
!python /content/detect_bccd.py \
  --data_folder /content/ssd/data_bccd \
  --trained_model /content/checkpoints/ssd300_bccd.pth \
  --label_map /content/utils_bccd.py \
  --min_score 0.2 \
  --max_overlap 0.35 \
  --top_k 35

# вычисляем метрики
!python /content/ssd/eval.py \
  --trained_model /content/checkpoints/ssd300_bccd.pth \
  --data_folder /content/ssd/data_bccd \
  --label_map /content/utils_bccd.py


/usr/bin/python3 /content/ssd/train.py --data_folder /content/ssd/data_bccd --label_map /content/utils_bccd.py --checkpoint /content/checkpoints/ssd300_bccd.pth --num_epochs 25 --batch_size 8 --validation
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Loaded base model.

/usr/local/lib/python3.11/dist-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' 

In [30]:
!zip -r hw7_project.zip /content/* -x "/content/sample_data/*"

  adding: content/bccd_raw/ (stored 0%)
  adding: content/bccd_raw/test.csv (deflated 78%)
  adding: content/bccd_raw/.git/ (stored 0%)
  adding: content/bccd_raw/.git/info/ (stored 0%)
  adding: content/bccd_raw/.git/info/exclude (deflated 28%)
  adding: content/bccd_raw/.git/config (deflated 32%)
  adding: content/bccd_raw/.git/description (deflated 14%)
  adding: content/bccd_raw/.git/refs/ (stored 0%)
  adding: content/bccd_raw/.git/refs/remotes/ (stored 0%)
  adding: content/bccd_raw/.git/refs/remotes/origin/ (stored 0%)
  adding: content/bccd_raw/.git/refs/remotes/origin/HEAD (stored 0%)
  adding: content/bccd_raw/.git/refs/tags/ (stored 0%)
  adding: content/bccd_raw/.git/refs/heads/ (stored 0%)
  adding: content/bccd_raw/.git/refs/heads/master (stored 0%)
  adding: content/bccd_raw/.git/HEAD (stored 0%)
  adding: content/bccd_raw/.git/objects/ (stored 0%)
  adding: content/bccd_raw/.git/objects/info/ (stored 0%)
  adding: content/bccd_raw/.git/objects/pack/ (stored 0%)
  adding

In [31]:
from google.colab import files
files.download('hw7_project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>